In [1]:
from qpe import *

from numpy.random import permutation

from time import time
from scipy.stats import gaussian_kde

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

matplotlib.rcParams['font.family']='serif'
matplotlib.rcParams['axes.unicode_minus']=False
matplotlib.rcParams.update({'font.size': 20})
matplotlib.rc('text', usetex=True)

In [2]:
def aux_info(dms, labels, n_layers, pars, n_copies=1, n_meas=0):

    n_inp = int(log2(len(dms[0])))
    n_tot = n_inp*n_copies
    d = 2**n_tot
    d_diff = 2**(n_tot - n_meas)
    
    if n_meas == 0:
        n_meas = n_tot
    
    pars_ans = pars[:-2**n_meas]
    pars_est = pars[-2**n_meas:]
    
    ansatz = hea_cry_rzrx(n_tot, n_layers, pars_ans)

    obs_u = ansatz.conj().T@kron_A_N(diag(pars_est), d_diff)@ansatz
    obs_u_sq = obs_u@obs_u
    
    # projs = [reduce(kron, [diag(line), eye(2**(n_tot - n_meas))]) for line in eye(2**n_meas)]
    # projs_u = [ansatz.conj().T@proj@ansatz for proj in projs]

    dms_cop = [reduce(kron, [dm]*n_copies) for dm in dms]
    
    expecs = []
    disps = []
    for dm in dms_cop:
        expec = trace(dm@obs_u).real
        disp = trace(dm@obs_u_sq).real - expec**2
        expecs.append(expec)
        disps.append(disp)
        
    
    return array(expecs), array(disps)

### Settings

In [3]:
n_inp = 2
n_test = 10000
mixed = True
marks = "neg"

dms_test, labels_test = gen_even_ent_data(n_test, n_inp=n_inp, mixed=mixed, marks=marks, eps=1e-3)
purities_test = [trace(dm@dm).real for dm in dms_test]

# Pass $T$

In [7]:
n_copies = 4
n_layers = 2
n_tot = n_inp*n_copies
n_meas = n_inp*n_copies

In [8]:
w_ls = 1e0
w_var = 1e-4
method = "BFGS"
options = {"maxiter": 5000}#, "maxfun": int(1e10)}##
T_min, T_max, T_step = 100, 1000, 100
n_train_list = arange(T_min, T_max + 1, T_step)

n_runs = 1

path = "/home/kardashin/Work/QPE/Results/Entanglement/Training_set_size/"
pref = "c=%d-m=%d-l=%d-w_ls=%.1f-w_var=%.4f-T=(%d,%d,%d)-%s" %(n_copies, n_meas, n_layers, w_ls, w_var, T_min, T_max, T_step, method)
path + pref

'/home/kardashin/Work/QPE/Results/Entanglement/Training_set_size/c=4-m=8-l=2-w_ls=1.0-w_var=0.0001-T=(100,1000,100)-BFGS'

In [9]:
# dms_train = np.load(path + "c=4-m=8-l=2-w_ls=1.0-w_var=0.0001-T=(100,1000,100)-BFGS-r=0-dms_train.npy")
# labels_train = np.load(path + "c=4-m=8-l=2-w_ls=1.0-w_var=0.0001-T=(100,1000,100)-BFGS-r=0-labels_train.npy")
# fvals = np.load(path + "c=4-m=8-l=2-w_ls=1.0-w_var=0.0001-T=(100,1000,100)-BFGS-r=0-Tr=100-fvals.npy")
# pars = np.load(path + "c=4-m=8-l=2-w_ls=1.0-w_var=0.0001-T=(100,1000,100)-BFGS-r=0-Tr=100-pars.npy")

In [ ]:
fvals_list = []
xf_list = []
nfev_list = []
time_list = []
error_test_list = []
dms_train_full_list = []
labels_train_full_list = []

for r in range(n_runs):
    print("run: %d" %(r + 1))

    dms_train_full, labels_train_full = gen_even_ent_data(n_train_list[-1], n_inp=n_inp, mixed=mixed, marks=marks)
    # shuffle
    perm = permutation(len(labels_train_full))
    dms_train_full = array(dms_train_full)[perm]
    labels_train_full = array(labels_train_full)[perm]
    dms_train_full_list.append(dms_train_full)
    labels_train_full_list.append(labels_train_full)

    file_name = path + pref + "-r=%d"%r
    np.save(file_name + "-dms_train", dms_train_full)
    np.save(file_name + "-labels_train", labels_train_full)

    x0 = array(concatenate([normal(pi, 1, 2*n_tot + (3*n_tot - 1)*n_layers), normal(0, 1, 2**n_meas)]))

    fvals_list_r = []
    xf_list_r = []
    nfev_list_r = []
    time_list_r = []
    error_test_list_r = []
    for n_train in n_train_list:
        print("\tT: %d" %n_train)
        
        dms_train = dms_train_full[:n_train]
        labels_train = labels_train_full[:n_train]

        file_name = path + pref + "-r=%d-Tr=%d"%(r, n_train)
        
        print("\t\tTraining")
        time_start = time()
        fvals, result = train(dms_train, labels_train, n_layers, 
                                  n_copies=n_copies, n_meas=n_meas, x0=x0,
                                  method=method, w_ls=w_ls, w_var=w_var, options=options,
                                  save_data=True, file_name=file_name, fvals=[])
        time_finish = time() - time_start
        print("\n\t\t\tFinished in", time_finish)
        print("\t\t\t", result.message)
        print("\t\t\tCost:", result.fun)
        
        print("\t\tTesting")
    
        expecs_test, disps_test = aux_info(dms_test, labels_test, n_layers, result.x, n_copies=n_copies, n_meas=n_meas)
        error_test = np.sum((expecs_test - labels_test)**2)
        print("\t\t\tTesting error:", error_test)

        fvals_list_r.append(fvals)
        xf_list_r.append(result.x)
        nfev_list_r.append(result.nfev)
        time_list_r.append(time_finish)
        error_test_list_r.append(error_test)

        np.save(file_name + "-fvals", array(fvals_list_r, dtype=object))
        np.save(file_name + "-pars", array(xf_list_r))
        np.save(file_name + "-nfev", array(nfev_list_r))
        np.save(file_name + "-time", array(time_list_r))
        np.save(file_name + "-error", array(error_test_list_r))

        x0 = result.x

    fvals_list.append(fvals_list_r)
    xf_list.append(xf_list_r)
    nfev_list.append(nfev_list_r)
    time_list.append(time_list_r)
    error_test_list.append(error_test_list_r)

    file_name = path + pref + "-r=%d"%r
    np.save(file_name + "-fvals", array(fvals_list_r, dtype=object))
    np.save(file_name + "-pars", array(xf_list_r))
    np.save(file_name + "-nfev", array(nfev_list_r))
    np.save(file_name + "-time", array(time_list_r))
    np.save(file_name + "-error", array(error_test_list_r))

run: 1
	T: 100
		Training
			Iteration: 3181 | Cost: 0.01296139 | Time passed: 130796 s
			Finished in 130796.88060045242
			 Optimization terminated successfully.
			Cost: 0.01296139491703775
		Testing
			Testing error: 80.82745245432615
	T: 200
		Training
			Iteration: 2535 | Cost: 0.05405531 | Time passed: 185544 s
			Finished in 185545.07245969772
			 Optimization terminated successfully.
			Cost: 0.054055309458374494
		Testing
			Testing error: 49.57710753148997
	T: 300
		Training
			Iteration: 1214 | Cost: 0.11975786 | Time passed: 133625 s
			Finished in 133625.25820827484
			 Optimization terminated successfully.
			Cost: 0.11975785777190334
		Testing
			Testing error: 52.99045262491971
	T: 400
		Training
			Iteration: 1345 | Cost: 0.23863024 | Time passed: 190549 s
			Finished in 190549.2892127037
			 Optimization terminated successfully.
			Cost: 0.23863023599866992
		Testing
			Testing error: 38.66625676791716
	T: 500
		Training
			Iteration: 1172 | Cost: 0.38120111 | Time p

In [ ]:
from scipy.optimize import curve_fit

# method = "lm"
# method = "trf"
method = "dogbox"

# def fit_func(x, a, b): return a*x**(-1/2) + b
# def fit_func(x, a, b): return (x**a)*b
def fit_func(x, a, b, c): return (x**b)*a + c

# popt, pcov = curve_fit(fit_func, n_train_list, np.mean(error_test_list, axis=0), method=method)#, bounds=bounds)
popt, pcov = curve_fit(fit_func, n_train_list, np.min(error_test_list, axis=0), method=method)#, bounds=bounds)

print(popt)
# plt.scatter(n_train_list, np.mean(error_test_list, axis=0))
plt.scatter(n_train_list, np.min(error_test_list, axis=0), marker="x")
plt.plot(n_train_list, fit_func(n_train_list, *popt))

In [ ]:
plt.errorbar(n_train_list, np.mean(error_test_list, axis=0), yerr=np.std(error_test_list, axis=0), linestyle="", marker="o", markersize=10, capsize=5, label="average")
plt.scatter(n_train_list, np.min(error_test_list, axis=0), marker="x", s=100, label="minimal")
# plt.plot(n_train_list, [1e6*n_train**(-4/3) for n_train in n_train_list], label="fit")
# plt.plot(n_train_list, fit_func(n_train_list, *popt), label=r"$%.3f\, T^{%.3f} + %.3f$" %tuple(popt))
plt.xticks(n_train_list[::2], n_train_list[::2])
plt.xlabel(r"$T$, training set size")
plt.ylabel(r"Testing error")
plt.yscale("log")
plt.grid()
plt.legend()
plt.show()

plt.errorbar(n_train_list, np.mean(time_list, axis=0), yerr=np.std(time_list, axis=0), marker="o", markersize=10, capsize=5)
plt.xticks(n_train_list[::2], n_train_list[::2])
plt.xlabel(r"$T$, training set size")
plt.ylabel(r"Training time")
plt.title(r"Average total time: %d seconds" %np.sum(np.mean(time_list, axis=0)))
# plt.yscale("log")
plt.grid()
plt.show()

plt.errorbar(n_train_list, np.mean(nfev_list, axis=0), yerr=np.std(nfev_list, axis=0), marker="o", markersize=10, capsize=5)
plt.xticks(n_train_list[::2], n_train_list[::2])
plt.xlabel(r"$T$, training set size")
plt.ylabel(r"Function evaluations")
plt.title(r"Total function evaluations: %d" %np.sum(np.mean(nfev_list, axis=0)))
# plt.yscale("log")
plt.grid()
plt.show()

# Continue run

In [8]:
n_inp = 2
n_copies = 4
n_layers = 2
n_tot = n_inp*n_copies
n_meas = n_inp*n_copies

w_ls = 1e0
w_var = 1e-4
method = "BFGS"
options = {"maxiter": 1500}#, "maxfun": int(1e10)}##
T_min, T_max_old, T_step = 100, 1000, 100
T_max = 1000
n_train_list = arange(T_min, T_max_old + 1, T_step)

r = 0


path = "/home/kardashin/Work/QPE/Results/Entanglement/Training_set_size/"
# path = "C:\\Users\\karda\\Work\\QPE\\Results\\Entanglement\\Training_set_size\\"
pref = "c=%d-m=%d-l=%d-w_ls=%.1f-w_var=%.4f-T=(%d,%d,%d)-%s" %(n_copies, n_meas, n_layers, w_ls, w_var, T_min, T_max_old, T_step, method)
path = path + pref + "/"
path

'/home/kardashin/Work/QPE/Results/Entanglement/Training_set_size/c=4-m=8-l=2-w_ls=1.0-w_var=0.0001-T=(100,1000,100)-BFGS'

In [16]:
file_name = path + pref + "-r=%d"%r

dms_train_full = np.load(file_name + "-dms_train.npy", allow_pickle=True)
labels_train_full = np.load(file_name + "-labels_train.npy", allow_pickle=True)

dms_train_add, labels_train_add = gen_even_ent_data(T_max - T_max_old, n_inp=n_inp, mixed=True, marks="neg")
perm = permutation(T_max - T_max_old)
dms_train_add = array(dms_train_add)[perm]
labels_train_add = array(labels_train_add)[perm]

dms_train_full = concatenate([dms_train_full, dms_train_add])
labels_train_full = concatenate([labels_train_full, labels_train_add])

In [39]:
fvals_list_r = list(np.load(file_name + "-fvals.npy", allow_pickle=True))
xf_list_r = list(np.load(file_name + "-pars.npy", allow_pickle=True))
nfev_list_r = list(np.load(file_name + "-nfev.npy", allow_pickle=True))
time_list_r = list(np.load(file_name + "-time.npy", allow_pickle=True))
error_test_list_r = list(np.load(file_name + "-error.npy", allow_pickle=True))

x0 = xf_list_r[-1]

In [44]:
path = "/home/kardashin/Work/QPE/Results/Entanglement/Training_set_size/"
pref = "c=%d-m=%d-l=%d-w_ls=%.1f-w_var=%.4f-T=(%d,%d,%d)-%s" %(n_copies, n_meas, n_layers, w_ls, w_var, T_min, T_max, T_step, method)
path = path + pref + "/"
path

'/home/kardashin/Work/QPE/Results/Entanglement/Training_set_size/c=4-m=8-l=2-w_ls=1.0-w_var=0.0001-T=(100,1000,100)-BFGS/'

In [25]:
np.save(path + pref + "-r=%d"%r + "-dms_train", dms_train_full)
np.save(path + pref + "-r=%d"%r + "-labels_train", labels_train_full)

In [53]:
for n_train in arange(T_max_old + T_step, T_max + 1, T_step):
    print("\tT: %d" %n_train)

    dms_train = dms_train_full[:n_train]
    labels_train = labels_train_full[:n_train]

    file_name = path + pref + "-r=%d-Tr=%d"%(r, n_train)

    print("\t\tTraining")
    time_start = time()
    fvals, result = train(dms_train, labels_train, n_layers, 
                              n_copies=n_copies, n_meas=n_meas, x0=x0,
                              method=method, w_ls=w_ls, w_var=w_var, options=options,
                              save_data=True, file_name=file_name)
    time_finish = time() - time_start
    print("\n\t\t\tFinished in", time_finish)
    print("\t\t\t", result.message)
    print("\t\t\tCost:", result.fun)

    x0 = result.x

    print("\t\tTesting")

    expecs_test, disps_test = aux_info(dms_test, labels_test, n_layers, result.x, n_copies=n_copies, n_meas=n_meas)
    error_test = np.sum((expecs_test - labels_test)**2)
    print("\t\t\tTesting error:", error_test)

    fvals_list_r.append(fvals)
    xf_list_r.append(result.x)
    nfev_list_r.append(result.nfev)
    time_list_r.append(time_finish)
    error_test_list_r.append(error_test)

    np.save(file_name + "-fvals", array(fvals_list_r, dtype=object))
    np.save(file_name + "-pars", array(xf_list_r))
    np.save(file_name + "-nfev", array(nfev_list_r))
    np.save(file_name + "-time", array(time_list_r))
    np.save(file_name + "-error", array(error_test_list_r))

	T: 900
		Training
			Iteration: 895 | Cost: 1.39423879 | Time passed: 272794 s
			Finished in 272794.3372476101
			 Optimization terminated successfully.
			Cost: 1.3942387869332553
		Testing
			Testing error: 25.122806917850244
	T: 1000
		Training
			Iteration: 528 | Cost: 1.60778138 | Time passed: 173741 s
			Finished in 173741.9537167549
			 Optimization terminated successfully.
			Cost: 1.6077813783072126
		Testing
			Testing error: 23.68100002332551


In [56]:
file_name = path + pref + "-r=%d"%r
np.save(file_name + "-fvals", array(fvals_list_r, dtype=object))
np.save(file_name + "-pars", array(xf_list_r))
np.save(file_name + "-nfev", array(nfev_list_r))
np.save(file_name + "-time", array(time_list_r))
np.save(file_name + "-error", array(error_test_list_r))